In [3]:
# If not already installed, do: pip install pandas fastparquet
import pandas as pd

URL_DATA = 'https://storage.data.gov.my/environment/air_pollution.parquet'

df = pd.read_parquet(URL_DATA)
if 'date' in df.columns: df['date'] = pd.to_datetime(df['date'])

print(df)

          date pollutant  concentration
0   2017-01-01        CO         0.5610
1   2017-02-01        CO         0.5300
2   2017-03-01        CO         0.5890
3   2017-04-01        CO         0.6620
4   2017-05-01        CO            NaN
..         ...       ...            ...
427 2022-08-01       SO2         0.0012
428 2022-09-01       SO2         0.0012
429 2022-10-01       SO2         0.0012
430 2022-11-01       SO2         0.0012
431 2022-12-01       SO2         0.0013

[432 rows x 3 columns]


In [12]:
# split to multiple csv for each pollutant
pollutants = df['pollutant'].unique()
for pollutant in pollutants:
    df[df['pollutant'] == pollutant].to_csv(f'../data/my_{pollutant}_data.csv', index=False)